# imports

In [ ]:
import sys
import os
PROJECT_ROOT = os.path.abspath("..")  # ajusta según tu estructura
sys.path.append(PROJECT_ROOT)

import json
from pathlib import Path
from xml.etree.ElementInclude import include

import numpy as np
import pandas as pd
import joblib
from torch.utils.data import DataLoader, Dataset, TensorDataset
from torch import nn

from machine_learning.data_collectors import (
    build_ml_dataframe,
    build_supervised_dataset,
    time_split_masks,
    purged_ts_cv_splits,
    TARGET_HORIZONS,
    TARGET_LOOKBACKS,
)

from machine_learning.evaluators import eval_regression, eval_regression_extended

from python_scripts.LLM_analysis.preprocess_store_database import get_connection
from database_tier1 import TARGET_STOCKS

from typing import Tuple, Dict, List, Optional, Union, Iterable
from sklearn.preprocessing import StandardScaler
from machine_learning.artifacts import load_mlp_artifact, predict_artifact_to_compare
from machine_learning.evaluators import calculate_deadzone
from machine_learning.artifacts import discover_artifacts_by_horizon

import torch
import random
from machine_learning.artifacts import predict_artifact_live

# load models by horizon

In [2]:
conn = get_connection()
symbols = TARGET_STOCKS

timeframe = "1Day"
# lookback = TARGET_LOOKBACKS[3]
horizon = TARGET_HORIZONS[2]

include_indicators = True

indicators_used = ['RSI_14', 'BBB_20_2.0', 'BBP_20_2.0', 'ATRr_14']

# indicators_used = ['SMA_20', 'EMA_50', 'RSI_14',
#        'MACD_12_26_9', 'MACDh_12_26_9', 'MACDs_12_26_9', 'BBL_20_2.0',
#        'BBM_20_2.0', 'BBU_20_2.0', 'BBB_20_2.0', 'BBP_20_2.0', 'ATRr_14']

include_economic_indicators = False
econ_indicator_names = ['CPI', 'UNEMPLOYMENT']

include_fmp = False
fmp_prefix = "fmp"
fmp_feature_names = []
keep_fmp_asof_date = False

base_feature_cols = ["open", "high", "low", "close", "volume", "trade_count", "vwap"] + indicators_used
horizon


20

In [3]:


artifacts = discover_artifacts_by_horizon(horizon)

for art in artifacts:
    print(art.get('run_dir'))

    

runs/cnn1d_1Day_lb252_h20_indicators0_econ0_fmp0_trial4_convchan64-64-64-64-64-64-64
runs/cnn1d_1Day_lb252_h20_indicators0_econ0_fmp0_trial7_convchan32-64-64-128-128-128
runs/mlp_1Day_lb120_h20_indicators0_econ0_fmp0_trial3_hs256-128_lr0.0001_do0.2_bs512
runs/mlp_1Day_lb120_h20_indicators0_econ0_fmp0_trial6_hs256-128_lr0.0001_do0.2_bs512
runs/mlp_1Day_lb120_h20_indicators1_econ0_fmp0_trial8_hs256-128_lr0.0001_do0.25_bs512
runs/mlp_1Day_lb252_h20_indicators0_econ0_fmp0_trial9_hs512-256-128_lr0.0001_do0.2_bs512
runs/mlp_1Day_lb252_h20_indicators1_econ0_fmp0_trial5_hs256-128_lr0.0001_do0.25_bs512
runs/mlp_1Day_lb60_h20_indicators1_econ0_fmp0_trial9_hs128-64_lr0.0001_do0.12_bs512


In [4]:



target_timestamps = [pd.to_datetime('2026-01-20 05:00:00+00:00')]

# chosen_run_dirs = [art.get('run_dir') for art in artifacts]
chosen_run_dirs = []
chosen_run_dirs = ['runs/cnn1d_1Day_lb252_h20_indicators0_econ0_fmp0_trial4_convchan64-64-64-64-64-64-64',
                   'runs/mlp_1Day_lb120_h20_indicators1_econ0_fmp0_trial8_hs256-128_lr0.0001_do0.25_bs512',
                   'runs/mlp_1Day_lb252_h20_indicators1_econ0_fmp0_trial5_hs256-128_lr0.0001_do0.25_bs512',
                   'runs/mlp_1Day_lb60_h20_indicators1_econ0_fmp0_trial9_hs128-64_lr0.0001_do0.12_bs512'
                   ]
metrics_by_model = {}

for run_dir in chosen_run_dirs:
    df_gen, meta = predict_artifact_to_compare(
        run_dir,
        timeframe="1Day",
        symbols=TARGET_STOCKS,
        start="2023-01-01",
    )
    df_gen = df_gen[df_gen['target_timestamp'].isin(target_timestamps)]
    meta = meta[meta['target_timestamp'].isin(target_timestamps)]
    y_true = df_gen['y_true'].to_numpy()
    y_pred = df_gen['y_pred'].to_numpy()
    metrics = eval_regression_extended(y_true, y_pred, deadzone=calculate_deadzone(horizon), meta=meta)
    metrics_by_model[run_dir] = metrics


ValueError: Faltan lag cols para secuencia. Ej: ['open_lag0_lag251', 'high_lag0_lag251', 'low_lag0_lag251', 'close_lag0_lag251', 'volume_lag0_lag251', 'trade_count_lag0_lag251', 'open_lag1_lag251', 'high_lag1_lag251', 'low_lag1_lag251', 'close_lag1_lag251']

## generate metrics

In [ ]:
from machine_learning.artifacts import predict_ridge_by_run_dir, predict_torch_by_run_dir

def metrics_matrix(metrics_by_model: dict[str, dict]) -> pd.DataFrame:
    mat = pd.DataFrame(metrics_by_model)  # index=metric, columns=model
    # ordena: N arriba, luego lo demás
    if "N" in mat.index:
        ordered = ["N", 'DailyRankIC_mean', 'DailyRankIC_frac_pos', 'QuantileSpread_sharpe', 'DailyIC_mean', 'HitRate(sign,deadzone)', 'AUC(Sign)', 'QuantileSpread_std', 'DailyRankIC_std']
        order = ordered + [i for i in mat.index if i not in ordered]
        mat = mat.loc[order]
    return mat

test_mat = metrics_matrix(metrics_by_model)
test_mat


,runs/mlp_1Day_lb120_h20_indicators1_econ0_fmp0_trial8_hs256-128_lr0.0001_do0.25_bs512
N,0.0
DailyRankIC_mean,NaN
DailyRankIC_frac_pos,NaN
QuantileSpread_sharpe,NaN
DailyIC_mean,NaN
"HitRate(sign,deadzone)",NaN
AUC(Sign),NaN
QuantileSpread_std,NaN
DailyRankIC_std,NaN
MAE,NaN


In [ ]:
from IPython.display import display
from matplotlib.colors import LinearSegmentedColormap

def style_metrics_by_row(
    mat: pd.DataFrame,
    exclude_rows=("N",),
    lower_is_better=(),   # e.g. ("QuantileSpread_std", "RMSE", ...)
    fmt="{:.4f}",
    row_limits: dict[str, tuple[float, float]] | None = None,  # <-- NUEVO
    clip: bool = True,                                       # <-- NUEVO
):
    mat_num = mat.copy().apply(pd.to_numeric, errors="coerce")

    # gmap: 0..1 por celda (0=peor/rojo, 1=mejor/verde)
    gmap = pd.DataFrame(index=mat_num.index, columns=mat_num.columns, dtype=float)

    row_limits = row_limits or {}

    for r in mat_num.index:
        row = mat_num.loc[r]

        # 1) Normalización con límites manuales si existen
        if r in row_limits:
            vmin, vmax = row_limits[r]
            denom = (vmax - vmin)
            if denom == 0:
                g = pd.Series(np.nan, index=row.index, dtype=float)
            else:
                g = (row - vmin) / denom
                if clip:
                    g = g.clip(0.0, 1.0)

        # 2) Si no hay límites, normaliza por min/max entre modelos (como antes)
        else:
            row_min = row.min()
            row_max = row.max()
            denom = (row_max - row_min)
            if pd.isna(denom) or denom == 0:
                g = pd.Series(np.nan, index=row.index, dtype=float)
            else:
                g = (row - row_min) / denom  # ya queda en 0..1

        # 3) Invertir escala si menor es mejor (después de normalizar)
        if r in lower_is_better:
            g = 1.0 - g

        gmap.loc[r] = g

    # (opcional) no colorear algunas filas
    rows_to_color = mat_num.index.difference(list(exclude_rows))

    # Colormap rojo → blanco → verde
    cmap = LinearSegmentedColormap.from_list(
        "red_white_green", ["#d73027", "#ffffff", "#1a9850"]
    )

    return (
        mat_num.style
            .format(fmt)
            .background_gradient(
                axis=None,
                cmap=cmap,
                gmap=gmap,
                subset=pd.IndexSlice[rows_to_color, :]  # type: ignore
            )
    )

row_limits = {
    "DailyRankIC_mean": (-0.02, 0.05),
    "DailyRankIC_frac_pos": (0.40, 0.60),
    "QuantileSpread_sharpe": (-2, 2.0),
    "DailyIC_mean": (-0.01, 0.02),
    "HitRate(sign,deadzone)": (0.4, 0.6),
    "AUC(Sign)": (0.4, 0.6),
}

styled = style_metrics_by_row(
    test_mat,
    exclude_rows=("N", "DailyIC_N", "N_deadzone", "DailyIC_tstat", "QuantileSpread_N",
                  "Conformal_avg_width(alpha=0.1)", "Conformal_avg_width(alpha=0.05)"),
    lower_is_better=("QuantileSpread_std", "DailyRankIC_std", "MAE", "MedianAE", "RMSE", "DailyIC_std"),
    row_limits=row_limits,
    clip=True,
)

display(styled)



,runs/cnn1d_1Day_lb252_h20_indicators0_econ0_fmp0_trial4_convchan64-64-64-64-64-64-64,runs/mlp_1Day_lb120_h20_indicators1_econ0_fmp0_trial8_hs256-128_lr0.0001_do0.25_bs512,runs/mlp_1Day_lb252_h20_indicators1_econ0_fmp0_trial5_hs256-128_lr0.0001_do0.25_bs512,runs/mlp_1Day_lb60_h20_indicators1_econ0_fmp0_trial9_hs128-64_lr0.0001_do0.12_bs512
N,82.0000,82.0000,82.0000,82.0000
DailyRankIC_mean,-0.0323,0.2314,-0.0916,0.1820
DailyRankIC_frac_pos,0.0000,1.0000,0.0000,1.0000
QuantileSpread_sharpe,nan,nan,nan,nan
DailyIC_mean,-0.0044,0.1692,-0.0753,0.1944
"HitRate(sign,deadzone)",0.6184,0.5789,0.5526,0.5526
AUC(Sign),0.5154,0.5923,0.4685,0.5931
QuantileSpread_std,nan,nan,nan,nan
DailyRankIC_std,nan,nan,nan,nan
MAE,0.0723,0.0740,0.0819,0.0761


## find models and compare metrics

In [ ]:
artifacts = discover_artifacts_by_horizon(20)

metrics_models = {}

for art in artifacts:
    metrics_models[art.get('run_dir')] = art.get('metrics').get('test')

test_mat = metrics_matrix(metrics_models)

styled = style_metrics_by_row(
    test_mat,
    exclude_rows=("N", "DailyIC_N", "N_deadzone", "DailyIC_tstat", "QuantileSpread_N",
                  "Conformal_avg_width(alpha=0.1)", "Conformal_avg_width(alpha=0.05)"),
    lower_is_better=("QuantileSpread_std", "DailyRankIC_std", "MAE", "MedianAE", "RMSE", "DailyIC_std"),
    row_limits=row_limits,
    clip=True,
)

display(styled)

,runs/cnn1d_1Day_lb252_h20_indicators0_econ0_fmp0_trial4_convchan64-64-64-64-64-64-64,runs/cnn1d_1Day_lb252_h20_indicators0_econ0_fmp0_trial7_convchan32-64-64-128-128-128,runs/mlp_1Day_lb120_h20_indicators0_econ0_fmp0_trial3_hs256-128_lr0.0001_do0.2_bs512,runs/mlp_1Day_lb120_h20_indicators0_econ0_fmp0_trial6_hs256-128_lr0.0001_do0.2_bs512,runs/mlp_1Day_lb120_h20_indicators1_econ0_fmp0_trial8_hs256-128_lr0.0001_do0.25_bs512,runs/mlp_1Day_lb252_h20_indicators0_econ0_fmp0_trial9_hs512-256-128_lr0.0001_do0.2_bs512,runs/mlp_1Day_lb252_h20_indicators1_econ0_fmp0_trial5_hs256-128_lr0.0001_do0.25_bs512,runs/mlp_1Day_lb60_h20_indicators1_econ0_fmp0_trial9_hs128-64_lr0.0001_do0.12_bs512
N,27744.0000,27744.0000,8802.0000,8802.0000,18642.0000,8310.0000,18478.0000,18642.0000
DailyRankIC_mean,0.0437,0.0168,-0.0325,0.0030,0.0284,0.0245,0.0279,0.0281
DailyRankIC_frac_pos,0.6932,0.5959,0.4722,0.5185,0.5833,0.5882,0.6150,0.6053
QuantileSpread_sharpe,7.5846,-0.2636,-4.8649,4.3896,-2.7172,-7.3258,0.8504,4.0485
DailyIC_mean,0.0583,0.0091,0.0259,0.0540,0.0108,-0.0199,0.0279,0.0426
"HitRate(sign,deadzone)",0.5555,0.5488,0.5924,0.5809,0.5182,0.5777,0.5328,0.5248
AUC(Sign),0.5236,0.5167,0.4790,0.4883,0.5127,0.5142,0.5127,0.5097
QuantileSpread_std,0.0683,0.0363,0.0591,0.0811,0.0932,0.1459,0.0822,0.0642
DailyRankIC_std,0.1054,0.0946,0.1437,0.1153,0.1122,0.1438,0.1294,0.1138
MAE,0.0645,0.0646,0.0639,0.0637,0.0686,0.0649,0.0691,0.0678


# todays prediction

In [20]:
from machine_learning.artifacts import predict_artifact_live


pred_live = predict_artifact_live(
    run_dir="runs/xgb_1Day_lb120_h20_indicators1_econ0_fmp0_md6_lr0.03_mcw5_seed42",
    select="latest",
    start="2023-01-01"
)

pred_live.sort_values("y_pred", ascending=False)


/home/aprohack/Desktop/all_folders/Investings_project/app/machine_learning/artifacts.py:3584: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(_rank_bucket)


,symbol,timestamp,target_timestamp_est,y_pred,horizon,timeframe,run_dir,model_name,close,pred_price,rank,rank_pct,bucket
47,MU,2026-01-30 05:00:00,2026-02-27 05:00:00,0.018151,20,1Day,runs/xgb_1Day_lb120_h20_indicators1_econ0_fmp0...,xgboost_regressor,414.88,422.479318,1,0.012195,long
30,INTU,2026-01-30 05:00:00,2026-02-27 05:00:00,0.011721,20,1Day,runs/xgb_1Day_lb120_h20_indicators1_econ0_fmp0...,xgboost_regressor,498.92,504.802274,2,0.024390,long
12,BTC,2026-01-30 05:00:00,2026-02-27 05:00:00,0.009685,20,1Day,runs/xgb_1Day_lb120_h20_indicators1_econ0_fmp0...,xgboost_regressor,37.08,37.440851,3,0.036585,long
65,UNH,2026-01-30 05:00:00,2026-02-27 05:00:00,0.009646,20,1Day,runs/xgb_1Day_lb120_h20_indicators1_econ0_fmp0...,xgboost_regressor,286.93,289.711262,26,0.317073,neutral
66,V,2026-01-30 05:00:00,2026-02-27 05:00:00,0.009646,20,1Day,runs/xgb_1Day_lb120_h20_indicators1_econ0_fmp0...,xgboost_regressor,321.83,324.949554,27,0.329268,neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,XLRE,2026-01-30 05:00:00,2026-02-27 05:00:00,0.008994,20,1Day,runs/xgb_1Day_lb120_h20_indicators1_econ0_fmp0...,xgboost_regressor,41.43,41.804322,78,0.951220,short
76,XLP,2026-01-30 05:00:00,2026-02-27 05:00:00,0.008901,20,1Day,runs/xgb_1Day_lb120_h20_indicators1_econ0_fmp0...,xgboost_regressor,83.51,84.256653,80,0.975610,short
35,KO,2026-01-30 05:00:00,2026-02-27 05:00:00,0.008901,20,1Day,runs/xgb_1Day_lb120_h20_indicators1_econ0_fmp0...,xgboost_regressor,74.81,75.478867,79,0.963415,short
46,MSFT,2026-01-30 05:00:00,2026-02-27 05:00:00,0.008689,20,1Day,runs/xgb_1Day_lb120_h20_indicators1_econ0_fmp0...,xgboost_regressor,430.29,434.045128,81,0.987805,short


In [19]:
from machine_learning.artifacts import predict_artifact_live


pred_live = predict_artifact_live(
    run_dir="runs/xgb_1Day_lb252_h60_indicators1_econ0_fmp0_md4_lr0.07_mcw5_seed42",
    select="latest",
    start="2023-01-01"
)

pred_live.sort_values("y_pred", ascending=False)


/home/aprohack/Desktop/all_folders/Investings_project/app/machine_learning/artifacts.py:3584: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(_rank_bucket)


,symbol,timestamp,target_timestamp_est,y_pred,horizon,timeframe,run_dir,model_name,close,pred_price,rank,rank_pct,bucket
47,MU,2026-01-30 05:00:00,2026-04-24 05:00:00,0.037184,60,1Day,runs/xgb_1Day_lb252_h60_indicators1_econ0_fmp0...,xgboost_regressor,414.88,430.597216,2,0.024390,long
30,INTU,2026-01-30 05:00:00,2026-04-24 05:00:00,0.037184,60,1Day,runs/xgb_1Day_lb252_h60_indicators1_econ0_fmp0...,xgboost_regressor,498.92,517.820968,1,0.012195,long
4,AMD,2026-01-30 05:00:00,2026-04-24 05:00:00,0.029703,60,1Day,runs/xgb_1Day_lb252_h60_indicators1_econ0_fmp0...,xgboost_regressor,236.73,243.867009,3,0.036585,long
59,SPY,2026-01-30 05:00:00,2026-04-24 05:00:00,0.023973,60,1Day,runs/xgb_1Day_lb252_h60_indicators1_econ0_fmp0...,xgboost_regressor,691.97,708.759134,60,0.731707,neutral
58,SBUX,2026-01-30 05:00:00,2026-04-24 05:00:00,0.023973,60,1Day,runs/xgb_1Day_lb252_h60_indicators1_econ0_fmp0...,xgboost_regressor,91.95,94.180965,59,0.719512,neutral
...,...,...,...,...,...,...,...,...,...,...,...,...,...
24,GLD,2026-01-30 05:00:00,2026-04-24 05:00:00,0.023973,60,1Day,runs/xgb_1Day_lb252_h60_indicators1_econ0_fmp0...,xgboost_regressor,444.95,455.745736,27,0.329268,neutral
23,GE,2026-01-30 05:00:00,2026-04-24 05:00:00,0.023973,60,1Day,runs/xgb_1Day_lb252_h60_indicators1_econ0_fmp0...,xgboost_regressor,306.79,314.233586,26,0.317073,neutral
22,FCX,2026-01-30 05:00:00,2026-04-24 05:00:00,0.023973,60,1Day,runs/xgb_1Day_lb252_h60_indicators1_econ0_fmp0...,xgboost_regressor,60.23,61.691349,25,0.304878,neutral
21,DIS,2026-01-30 05:00:00,2026-04-24 05:00:00,0.023973,60,1Day,runs/xgb_1Day_lb252_h60_indicators1_econ0_fmp0...,xgboost_regressor,112.80,115.536845,24,0.292683,neutral


In [18]:

from machine_learning.artifacts import predict_artifact_live


pred_live = predict_artifact_live(
    run_dir="runs/cnn1d_1Day_lb252_h20_indicators0_econ0_fmp0_trial4_convchan64-64-64-64-64-64-64",
    select="latest",
    start="2023-01-01"
)

pred_live.sort_values("y_pred", ascending=False)
pred_live.columns


family: torch
model name: cnn1d_regressor


/home/aprohack/Desktop/all_folders/Investings_project/app/machine_learning/artifacts.py:3584: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(_rank_bucket)


Index(['symbol', 'timestamp', 'target_timestamp_est', 'y_pred', 'horizon',
       'timeframe', 'run_dir', 'model_name', 'close', 'pred_price', 'rank',
       'rank_pct', 'bucket'],
      dtype='object')

In [17]:

from machine_learning.artifacts import predict_artifact_live


pred_live = predict_artifact_live(
    run_dir="runs/cnn1d_1Day_lb252_h60_indicators0_econ0_fmp0_trial0_convchan32-64-64",
    select="latest",
    start="2023-01-01"
)

pred_live.sort_values("y_pred", ascending=False)


family: torch
model name: cnn1d_regressor


/home/aprohack/Desktop/all_folders/Investings_project/app/machine_learning/artifacts.py:3584: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(_rank_bucket)


,symbol,timestamp,target_timestamp_est,y_pred,horizon,timeframe,run_dir,model_name,close,pred_price,rank,rank_pct,bucket
65,UNH,2026-01-30 05:00:00,2026-04-24 05:00:00,0.187615,60,1Day,runs/cnn1d_1Day_lb252_h60_indicators0_econ0_fm...,cnn1d_regressor,286.93,346.143576,1,0.012195,long
25,GOOGL,2026-01-30 05:00:00,2026-04-24 05:00:00,0.175409,60,1Day,runs/cnn1d_1Day_lb252_h60_indicators0_econ0_fm...,cnn1d_regressor,338.00,402.805955,2,0.024390,long
9,AVGO,2026-01-30 05:00:00,2026-04-24 05:00:00,0.173941,60,1Day,runs/cnn1d_1Day_lb252_h60_indicators0_econ0_fm...,cnn1d_regressor,331.30,394.242008,3,0.036585,long
0,AAPL,2026-01-30 05:00:00,2026-04-24 05:00:00,0.169799,60,1Day,runs/cnn1d_1Day_lb252_h60_indicators0_econ0_fm...,cnn1d_regressor,259.48,307.500979,4,0.048780,long
7,AMZN,2026-01-30 05:00:00,2026-04-24 05:00:00,0.156847,60,1Day,runs/cnn1d_1Day_lb252_h60_indicators0_econ0_fm...,cnn1d_regressor,239.30,279.937209,5,0.060976,long
...,...,...,...,...,...,...,...,...,...,...,...,...,...
64,TXN,2026-01-30 05:00:00,2026-04-24 05:00:00,-0.090229,60,1Day,runs/cnn1d_1Day_lb252_h60_indicators0_econ0_fm...,cnn1d_regressor,215.55,196.952684,78,0.951220,short
53,PEP,2026-01-30 05:00:00,2026-04-24 05:00:00,-0.091811,60,1Day,runs/cnn1d_1Day_lb252_h60_indicators0_econ0_fm...,cnn1d_regressor,153.63,140.153217,79,0.963415,short
45,MS,2026-01-30 05:00:00,2026-04-24 05:00:00,-0.102140,60,1Day,runs/cnn1d_1Day_lb252_h60_indicators0_econ0_fm...,cnn1d_regressor,182.80,165.050646,80,0.975610,short
15,COST,2026-01-30 05:00:00,2026-04-24 05:00:00,-0.104638,60,1Day,runs/cnn1d_1Day_lb252_h60_indicators0_econ0_fm...,cnn1d_regressor,940.25,846.836825,81,0.987805,short


# tcn

In [ ]:


# puedes pasar el root run_dir o el subdir final_model; se auto-resuelve a final_model
pred_live = predict_artifact_live(
    run_dir="runs/final_model_tcn/final_model",   # o ".../final_model"
    select="latest",
)

# ejemplo: ver top scores SOLO para horizon=20
pred_live[pred_live["horizon"] == 20].sort_values("y_pred", ascending=False)
# pred_live
pred_live.columns

/home/aprohack/Desktop/all_folders/Investings_project/app/investenv/lib/python3.12/site-packages/torch/nn/utils/weight_norm.py:144: FutureWarning: `torch.nn.utils.weight_norm` is deprecated in favor of `torch.nn.utils.parametrizations.weight_norm`.
  WeightNorm.apply(module, name, dim)
/home/aprohack/Desktop/all_folders/Investings_project/app/investenv/lib/python3.12/site-packages/torch/cuda/__init__.py:182: UserWarning: CUDA initialization: Unexpected error from cudaGetDeviceCount(). Did you run some cuda functions before calling NumCudaDevices() that might have already set an error? Error 804: forward compatibility was attempted on non supported HW (Triggered internally at /pytorch/c10/cuda/CUDAFunctions.cpp:119.)
  return torch._C._cuda_getDeviceCount() > 0
/home/aprohack/Desktop/all_folders/Investings_project/app/machine_learning/artifacts.py:3582: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas 

Index(['symbol', 'timestamp', 'target_timestamp_est', 'y_pred', 'horizon',
       'timeframe', 'run_dir', 'model_name', 'close', 'pred_price', 'rank',
       'rank_pct', 'bucket'],
      dtype='object')

In [27]:
import numpy as np
import pandas as pd
from typing import Dict, Optional

def combine_model_preds_common_day(
    preds_by_model: Dict[str, pd.DataFrame],
    *,
    horizon: int,
    q: float = 0.1,
    how: str = "inner",           # inner = solo símbolos comunes; outer = unión
    day_freq: str = "D",          # "D" para 1Day
    weights: Optional[Dict[str, float]] = None,
    require_at_least: int = 1,    # mínimo de modelos con pred para mantener fila (si how="outer")
) -> pd.DataFrame:
    if weights is None:
        weights = {}

    # 1) normaliza y filtra horizon
    per_model = {}
    day_sets = []
    for name, df in preds_by_model.items():
        d = df.copy()
        d["symbol"] = d["symbol"].astype(str)
        d["timestamp"] = pd.to_datetime(d["timestamp"], errors="coerce")

        d = d[d["horizon"].astype(int) == int(horizon)].copy()
        if len(d) == 0:
            per_model[name] = d
            day_sets.append(set())
            continue

        # normaliza a "día"
        d["ts_day"] = d["timestamp"].dt.floor(day_freq)

        # quédate con 1 fila por símbolo para ese día (si hay duplicados)
        # (no escogemos aún el día: lo haremos después)
        per_model[name] = d
        day_sets.append(set(d["ts_day"].dropna().unique()))

    # 2) último día común
    common_days = set.intersection(*day_sets) if day_sets else set()
    if not common_days:
        # Debug útil para entender por qué no intersecta
        info = []
        for name, d in per_model.items():
            if len(d) == 0:
                info.append((name, 0, None, None))
            else:
                info.append((name, len(d), d["ts_day"].min(), d["ts_day"].max()))
        raise ValueError(
            "No hay ningún día (timestamp) común entre modelos para ese horizon.\n"
            "Resumen por modelo (rows, min_day, max_day):\n" +
            "\n".join([f"- {n}: rows={r}, min={mn}, max={mx}" for (n,r,mn,mx) in info]) +
            "\nSolución: usa el mismo timeframe/start/end/symbols en todos los predict_artifact_live "
            "o cambia select/tail_n para forzar el mismo día."
        )

    ts_day_common = max(common_days)

    # 3) filtra cada modelo a ese día y reduce columnas
    frames = []
    for name, d in per_model.items():
        d = d[d["ts_day"] == ts_day_common].copy()
        # si aún hay múltiples timestamps intradía, quédate con el último por símbolo
        d = d.sort_values(["symbol", "timestamp"]).groupby("symbol", as_index=False).tail(1)

        keep = ["symbol", "ts_day", "timestamp", "close", "y_pred", "rank_pct", "bucket", "pred_price"]
        keep = [c for c in keep if c in d.columns]
        d = d[keep].copy()

        # renombra por modelo (guardamos timestamp por modelo por si difiere)
        ren = {}
        for c in keep:
            if c in ("symbol", "ts_day"):
                continue
            ren[c] = f"{c}__{name}"
        d = d.rename(columns=ren)

        frames.append(d)

    # 4) merge por symbol + ts_day
    out = frames[0]
    for f in frames[1:]:
        out = out.merge(f, on=["symbol", "ts_day"], how=how)

    # 5) ensemble sobre rank_pct (lo más comparable)
    rank_cols = [c for c in out.columns if c.startswith("rank_pct__")]
    if not rank_cols:
        raise ValueError("No hay rank_pct__* en la tabla resultante. Incluye rank_pct en las preds.")

    W = np.array([float(weights.get(c.split("__", 1)[1], 1.0)) for c in rank_cols], dtype=float)
    W = np.where(np.isfinite(W) & (W > 0), W, 1.0)

    R = out[rank_cols].to_numpy(dtype=float)
    mask = np.isfinite(R)
    Wm = W.reshape(1, -1) * mask
    denom = Wm.sum(axis=1)
    denom = np.where(denom == 0, np.nan, denom)

    out["models_available"] = mask.sum(axis=1).astype(int)
    if how == "outer":
        out = out[out["models_available"] >= int(require_at_least)].copy()

    out["ensemble_rank_pct"] = (R * W.reshape(1, -1) * mask).sum(axis=1) / denom
    out["agreement_std_rank_pct"] = np.nanstd(R, axis=1)

    q = float(q)
    out["ensemble_bucket"] = np.where(
        out["ensemble_rank_pct"] <= q, "long",
        np.where(out["ensemble_rank_pct"] >= (1.0 - q), "short", "neutral")
    )

    # señal: positivo = compra, negativo = venta (por rank)
    out["ensemble_signal"] = 0.5 - out["ensemble_rank_pct"]

    # votos si existen buckets por modelo
    bucket_cols = [c for c in out.columns if c.startswith("bucket__")]
    if bucket_cols:
        b = out[bucket_cols].astype(str)
        out["votes_long"] = (b == "long").sum(axis=1)
        out["votes_short"] = (b == "short").sum(axis=1)
        out["votes_neutral"] = (b == "neutral").sum(axis=1)
        out["vote_margin"] = out["votes_long"] - out["votes_short"]

    return out.sort_values(["ensemble_rank_pct", "agreement_std_rank_pct"], ascending=[True, True]).reset_index(drop=True)


# join all predictions

In [30]:
preds = {
    "xgb1": predict_artifact_live(
        run_dir="runs/xgb_1Day_lb120_h20_indicators1_econ0_fmp0_md6_lr0.03_mcw5_seed42",
        select="latest",
        start="2023-01-01"
    ),
    # "xgb2": predict_artifact_live(
    #     run_dir="runs/xgb_1Day_lb252_h60_indicators1_econ0_fmp0_md4_lr0.07_mcw5_seed42",
    #     select="latest",
    #     start="2023-01-01"
    # ),
    "cnn1": predict_artifact_live(
        run_dir="runs/cnn1d_1Day_lb252_h20_indicators0_econ0_fmp0_trial4_convchan64-64-64-64-64-64-64",
        select="latest",
        start="2023-01-01"
    ),
    # "cnn2": predict_artifact_live(
    #     run_dir="runs/cnn1d_1Day_lb252_h60_indicators0_econ0_fmp0_trial0_convchan32-64-64",
    #     select="latest",
    #     start="2023-01-01"
    # ),
    "tcn":  predict_artifact_live(
        run_dir="runs/final_model_tcn/final_model",
        select="latest",
        start="2023-01-01"
    ),
}
tbl = combine_model_preds_common_day(
    preds,
    horizon=20,
    how="inner",
    require_at_least=3,  # exige al menos 3 de 5 modelos
    q=0.1
)
tbl
# # Top compras
# tbl.sort_values(["ensemble_rank_pct", "agreement_std_rank_pct"], ascending=[True, True]).head(30)

# # Top ventas
# tbl.sort_values(["ensemble_rank_pct", "agreement_std_rank_pct"], ascending=[False, True]).head(30)


/home/aprohack/Desktop/all_folders/Investings_project/app/machine_learning/artifacts.py:3582: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(_rank_bucket)
/home/aprohack/Desktop/all_folders/Investings_project/app/machine_learning/artifacts.py:3582: FutureWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  .apply(_rank_bucket)
/home/aprohack/Desktop/all_folders/Investings_project/app/investenv/lib/python3.12/site-packages/t

,symbol,ts_day,timestamp__xgb1,close__xgb1,y_pred__xgb1,rank_pct__xgb1,bucket__xgb1,pred_price__xgb1,timestamp__cnn1,close__cnn1,...,pred_price__tcn,models_available,ensemble_rank_pct,agreement_std_rank_pct,ensemble_bucket,ensemble_signal,votes_long,votes_short,votes_neutral,vote_margin
0,MU,2026-01-30,2026-01-30 05:00:00,414.88,0.018151,0.012195,long,422.479318,2026-01-30 05:00:00,414.88,...,671.008227,3,0.065041,0.051097,long,0.434959,2,0,1,2
1,AMZN,2026-01-30,2026-01-30 05:00:00,239.30,0.009646,0.060976,long,241.619576,2026-01-30 05:00:00,239.30,...,250.525093,3,0.081301,0.056619,long,0.418699,2,0,1,2
2,QQQ,2026-01-30,2026-01-30 05:00:00,621.87,0.009646,0.256098,neutral,627.897894,2026-01-30 05:00:00,621.87,...,716.894310,3,0.166667,0.069938,neutral,0.333333,1,0,2,1
3,INTU,2026-01-30,2026-01-30 05:00:00,498.92,0.011721,0.024390,long,504.802274,2026-01-30 05:00:00,498.92,...,514.296872,3,0.199187,0.139520,neutral,0.300813,1,0,2,1
4,TSLA,2026-01-30,2026-01-30 05:00:00,430.41,0.009646,0.292683,neutral,434.582039,2026-01-30 05:00:00,430.41,...,799.725372,3,0.199187,0.115407,neutral,0.300813,1,0,2,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
77,META,2026-01-30,2026-01-30 05:00:00,716.50,0.008368,1.000000,short,722.520811,2026-01-30 05:00:00,716.50,...,622.589237,3,0.752033,0.284495,neutral,-0.252033,0,2,1,-2
78,XLI,2026-01-30,2026-01-30 05:00:00,165.44,0.009095,0.902439,short,166.951503,2026-01-30 05:00:00,165.44,...,152.720056,3,0.796748,0.102194,neutral,-0.296748,0,1,2,-1
79,PEP,2026-01-30,2026-01-30 05:00:00,153.63,0.009095,0.829268,neutral,155.033604,2026-01-30 05:00:00,153.63,...,149.464444,3,0.800813,0.135921,neutral,-0.300813,0,1,2,-1
80,CVX,2026-01-30,2026-01-30 05:00:00,176.90,0.009095,0.670732,neutral,178.516205,2026-01-30 05:00:00,176.90,...,167.148837,3,0.821138,0.135921,neutral,-0.321138,0,1,2,-1


In [26]:
def summarize_pred(df, name, horizon=20):
    d = df.copy()
    d["timestamp"] = pd.to_datetime(d["timestamp"], errors="coerce")
    d = d[d["horizon"].astype(int) == int(horizon)].copy()
    if len(d) == 0:
        print(name, "-> 0 filas para horizon", horizon)
        return
    print(
        f"{name}: rows={len(d):,} symbols={d['symbol'].nunique():,} "
        f"ts_min={d['timestamp'].min()} ts_max={d['timestamp'].max()} "
        f"ts_nunique={d['timestamp'].nunique():,}"
    )
    print(d["timestamp"].value_counts().head(5))

for name, dfm in preds.items():
    summarize_pred(dfm, name, horizon=20)


xgb1: rows=82 symbols=82 ts_min=2026-01-30 05:00:00 ts_max=2026-01-30 05:00:00 ts_nunique=1
timestamp
2026-01-30 05:00:00    82
Name: count, dtype: int64
xgb2 -> 0 filas para horizon 20
cnn1: rows=82 symbols=82 ts_min=2026-01-30 05:00:00 ts_max=2026-01-30 05:00:00 ts_nunique=1
timestamp
2026-01-30 05:00:00    82
Name: count, dtype: int64
cnn2 -> 0 filas para horizon 20
tcn: rows=82 symbols=82 ts_min=2026-01-30 05:00:00 ts_max=2026-01-30 05:00:00 ts_nunique=1
timestamp
2026-01-30 05:00:00    82
Name: count, dtype: int64


In [25]:
df_gen, meta = predict_artifact_to_compare(
    run_dir="runs/tcn_walkforward_2026-01-26_....",  # o .../final_model
    timeframe="1Day",
    symbols=TARGET_STOCKS,
    start="2023-01-01",
)

for h in (5, 20, 60):
    df_h = df_gen[df_gen["horizon"] == h]
    meta_h = meta[meta["horizon"] == h]
    metrics = eval_regression_extended(
        df_h["y_true"].to_numpy(),
        df_h["y_pred"].to_numpy(),
        meta=meta_h,
        deadzone=calculate_deadzone(h),
    )
    print(h, metrics["DailyIC_mean"], metrics["RMSE"])


FileNotFoundError: No encuentro artifact en runs/tcn_walkforward_2026-01-26_..... Esperaba model.pt/pipeline.joblib aquí o en subdir final_model/.

In [ ]:
target_timestamps = [pd.to_datetime('2026-01-20 05:00:00+00:00')]

h = 20
df_h = df_gen[(df_gen["horizon"] == h) & (df_gen["target_timestamp"].isin(target_timestamps))]
meta_h = meta[(meta["horizon"] == h) & (meta["target_timestamp"].isin(target_timestamps))]

metrics = eval_regression_extended(df_h["y_true"].to_numpy(), df_h["y_pred"].to_numpy(), meta=meta_h)


In [ ]:
from machine_learning.artifacts import predict_artifact_live

pred_live = predict_artifact_live(
    run_dir="runs/tcn_walkforward_2026-01-26_....",   # root o .../final_model
    select="latest",
)

# OJO: el TCN devuelve 3 horizontes => filtra:
pred_live[pred_live["horizon"] == 20].sort_values("y_pred", ascending=False)


In [ ]:
metrics_by_model = {}

for run_dir in chosen_run_dirs:
    df_gen, meta = predict_artifact_to_compare(
        run_dir,
        timeframe="1Day",
        symbols=TARGET_STOCKS,
        start="2023-01-01",
    )

    # compara horizon=20 para TODOS (los single horizon tendrán solo ese)
    h = 20
    df_h = df_gen[df_gen["horizon"] == h]
    meta_h = meta[meta["horizon"] == h]

    y_true = df_h["y_true"].to_numpy()
    y_pred = df_h["y_pred"].to_numpy()
    metrics = eval_regression_extended(y_true, y_pred, deadzone=calculate_deadzone(h), meta=meta_h)
    metrics_by_model[run_dir] = metrics


## ultimas 20 predicciones

In [ ]:
pred_tail = predict_artifact_live(
    run_dir="runs/mlp_1Day_lb120_h20_indicators1_econ0_fmp0_trial8_hs256-128_lr0.0001_do0.25_bs512",
    select="tail",
    tail_n=20,
)
pred_tail


,symbol,timestamp,y_pred,target_timestamp_est,horizon,timeframe,run_dir,model_name,close,pred_price,rank,rank_pct,bucket
0,AAPL,2025-12-17 05:00:00+00:00,-0.010454,2026-01-14 05:00:00+00:00,20,1Day,runs/mlp_1Day_lb120_h20_indicators1_econ0_fmp0...,mlp_regressor,271.84,269.012873,64.0,0.780488,neutral
1,ABBV,2025-12-17 05:00:00+00:00,-0.003030,2026-01-14 05:00:00+00:00,20,1Day,runs/mlp_1Day_lb120_h20_indicators1_econ0_fmp0...,mlp_regressor,224.31,223.631263,52.0,0.634146,neutral
2,ADBE,2025-12-17 05:00:00+00:00,-0.015146,2026-01-14 05:00:00+00:00,20,1Day,runs/mlp_1Day_lb120_h20_indicators1_econ0_fmp0...,mlp_regressor,354.66,349.328722,67.0,0.817073,neutral
3,AMAT,2025-12-17 05:00:00+00:00,0.004362,2026-01-14 05:00:00+00:00,20,1Day,runs/mlp_1Day_lb120_h20_indicators1_econ0_fmp0...,mlp_regressor,248.27,249.355332,39.0,0.475610,neutral
4,AMD,2025-12-17 05:00:00+00:00,0.080695,2026-01-14 05:00:00+00:00,20,1Day,runs/mlp_1Day_lb120_h20_indicators1_econ0_fmp0...,mlp_regressor,198.11,214.759105,2.0,0.024390,long
...,...,...,...,...,...,...,...,...,...,...,...,...,...
1635,XLRE,2026-01-15 05:00:00+00:00,-0.023410,2026-02-12 05:00:00+00:00,20,1Day,runs/mlp_1Day_lb120_h20_indicators1_econ0_fmp0...,mlp_regressor,41.70,40.735129,71.0,0.865854,neutral
1636,XLU,2026-01-15 05:00:00+00:00,-0.024653,2026-02-12 05:00:00+00:00,20,1Day,runs/mlp_1Day_lb120_h20_indicators1_econ0_fmp0...,mlp_regressor,43.61,42.548027,72.0,0.878049,neutral
1637,XLV,2026-01-15 05:00:00+00:00,-0.007339,2026-02-12 05:00:00+00:00,20,1Day,runs/mlp_1Day_lb120_h20_indicators1_econ0_fmp0...,mlp_regressor,156.96,155.812349,50.0,0.609756,neutral
1638,XLY,2026-01-15 05:00:00+00:00,-0.018683,2026-02-12 05:00:00+00:00,20,1Day,runs/mlp_1Day_lb120_h20_indicators1_econ0_fmp0...,mlp_regressor,122.70,120.428931,66.0,0.804878,neutral
